In [51]:
using TSSOS
using LinearAlgebra
using QuantumOptics
using DynamicPolynomials
using QuadGK
using JuMP
using Random
using NLopt

## Define the quantum system

In [52]:
# We look at a typical Hamiltonian

#Drift Hamiltonian

H0 = [
    0 0 0;
    0 0.515916 0;
    0 0 1
];

H0 ./= norm(H0, Inf)

# Control Hamiltonian
V = [
    0 0.707107 0;
    0.707107 0 1;
    0 1 0
]

V ./= norm(V, Inf);

# final time
const T = 0.5
m =2
dt = T/m

0.25

In [53]:
H0*V-V*H0

3×3 Matrix{Float64}:
 0.0       -0.364808   0.0
 0.364808   0.0       -0.484084
 0.0        0.484084   0.0

## Useful functions with polynomials

In [54]:
function ∫(p::AbstractPolynomial, x::PolyVar, x_lower, x_upper)
    
    # get the index of the variable of integration
    ind_x = indexin([x], variables(p))[1]
        
    if isnothing(ind_x)
        # integration valuable is not found among vars
        return p * (x_upper - x_lower)
    end
    
    # get the indefinite integral
    int_p = sum(
        term * x * 1 // (exponents(term)[ind_x] + 1) for term in terms(p)
        init = 0 * x
    )
            
    # get the definite integral
    subs(int_p, x=>x_upper) - subs(int_p, x=>x_lower)
end

function ∫(M::AbstractMatrix, x::PolyVar, x_lower, x_upper)
   map(z -> ∫(z, x, x_lower, x_upper), M) 
end

function real_poly(p::Polynomial)
    #=
    Real part of the polynomial
    =#
    sum(
        real(c) * m for (c, m) in zip(coefficients(p), monomials(p))# if ~isapproxzero(abs(c))
    )
end

function square_frobenius_norm(M::AbstractArray)
    #=
    Square of the Frobenius norm of a matrix
    =#
    real_poly(sum(z' * z for z in M))
end

function square_frobenius_norm2(M::AbstractArray)
    #=
    Square of the Frobenius norm of a matrix
    =#
    sum(z' * z for z in M)
end

square_frobenius_norm2 (generic function with 1 method)

## Magnus expansion

In [55]:
@polyvar x[1:m]
@polyvar t[1:3]

function u(t, x)
    # the polynomial shape for control
    sum(x[n] * t^(n - 1) for n = 1:length(x))
end

function A(t, x,)
    #=
    The generator of motion entering the Magnus expansion
    =#
    (H0 + V * u(t, x)) / im
end

function commutator(a, b)
    a * b - b * a
end 

M = [-im * dt * (H0 + x[i] * V) for i in 1:m];

X = sum(M) ## first term

function commutator(A, B)
        return A * B - B * A
end

## second term
for i in 1:m
    for j in i:m
        X += 0.5 * commutator(M[i], M[j])
    end
end


## third term
for i in 1:m
    for j in i:m
        for k in j:m
        X += 1/6 * (commutator(M[i],commutator(M[j],M[k]))+commutator(M[k],commutator(M[j],M[i])))
        end
    end
end

## third term
for i in 1:m
    for j in i:m
        for k in j:m
            for l in k:m
                X += 1/12 *( commutator(commutator(commutator(M[i],M[j]),M[k]),M[l]) + commutator(M[i],commutator(commutator(M[j],M[k]),M[l])) + commutator(M[i],commutator(M[j],commutator(M[k],M[l]))) + commutator(M[j],commutator(M[k],commutator(M[l],M[i])))  )
            end
        
        end
    end
end

## Get random unitary target and take logarithm

In [56]:
function get_unitary(x::AbstractArray)
    #=
    Get the unitary given the coefficients for the polynomial control
    =#
    prod(exp(-im *dt* (H0 + xx * V)) for xx in x)
end

get_unitary (generic function with 1 method)

In [57]:
function logarithm_mat(U)
    F= eigen(U)
    Z = F.vectors
    L = Diagonal(F.values)
    n= 0
    theta_values = [log(λ) for λ in F.values]
    K = Diagonal(theta_values + 2*pi*im*n*[1,1,1])
    Z*K*inv(Z)
end

logarithm_mat (generic function with 1 method)

## Functions to compare

In [58]:
function local_minimize(obj::AbstractPolynomial, init_x::AbstractArray)
    #=
    Perform local minimization of obj polynomial using init_x as initial guess
    =#
    vars = variables(obj)

    @assert length(vars) == length(init_x)
    
    function g(a...)
        # Converting polynomial expression to function to be minimize
        obj(vars => a)
    end
    
    model = Model(NLopt.Optimizer)

    set_optimizer_attribute(model, "algorithm", :LD_MMA)

    set_silent(model)
    @variable(model, y[1:length(vars)])

    # set initial guess
    for (var, init_val) in zip(y, init_x)
        set_start_value(var, init_val)
    end

    register(model, :g, length(y), g; autodiff = true)
    @NLobjective(model, Min, g(y...))
    JuMP.optimize!(model)

    map(value, y)
end

local_minimize (generic function with 1 method)

## TSSOS

In [59]:
@time begin
    
n_samples = 100
Random.seed!(6292022)

# randomly generate the coefficients for the polynomial control 
exact_x = -1 .+ 2 * rand(length(x) * n_samples)
exact_x = reshape(exact_x, (length(x), n_samples))

# The values of objective functions for exact x
obj_exact_x = zeros(n_samples)
    
# The array of target unitaries synthesized by the control x
U_targets = zeros(ComplexF64, n_samples, size(H0)...)

# the polynomial objective at min_x
glob_obj_min_x = zeros(n_samples)

# The global minimum via TSSOS library
tssos_glob_obj_min = zeros(n_samples)

# Frobenius norm difference between target and obtained unitaries
norm_U_target_minus_obtained = zeros(n_samples)

# Frobenius norm difference between target and the truncated Magnus expansion
norm_U_target_minus_expΩ_exact_x = zeros(n_samples)
norm_U_target_minus_expΩ_min_x = zeros(n_samples)

# The normalised overlap of the evolution and the target 
f_PSU = zeros(n_samples) 

# Convergence test for the Magnus expansion for the exact control (convergence_test_exact_x < 1)
convergence_test_exact_x = zeros(n_samples)


# Convergence test for the Magnus expansion for the obtained control (convergence_test_min_x < 1)
convergence_test_min_x = zeros(n_samples)

norm_diff_x = zeros(n_samples)

Threads.@threads for i=1:n_samples
    
    # target unitray
    U_targets[i, :, :] = U_target = get_unitary(exact_x[:, i])
        
    # get the polynomial objective function
    obj = square_frobenius_norm(
        X - logarithm_mat(U_targets[i,:,:])
    )
    
    # save the value of objective function for exact x
    obj_exact_x[i] = obj(exact_x[:, i])
  
    
    # Get the global minimum via TSSOS library
    opt,sol,data = tssos_first(obj, variables(obj); QUIET = true, solution = true)
    
    previous_sol = sol
    previous_opt = opt
    
    while ~isnothing(sol)
        previous_sol = sol
        previous_opt = opt
            
        opt,sol,data = tssos_higher!(data; QUIET = true, solution = true)
    end
    tssos_glob_obj_min[i] = previous_opt
    min_x = previous_sol
    
    # refine the estimate by local minimization
    #min_x = local_minimize(obj, min_x)
    
    # the polynomial objective at min_x
    glob_obj_min_x[i] = tssos_glob_obj_min[i]
        
    # get the Frobenius norm difference between target and obtained unitaries
    U_star = get_unitary(min_x)
    norm_U_target_minus_obtained[i] = norm(U_target - U_star)

    # The normalised overlap of the evolution and the target 
    f_PSU[i] = abs(tr(U_target' * U_star)) / size(U_star)[1]
        
end

end

*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS **********************

In [60]:
using HDF5

filename = "results_$(m).hdf5"
h5open(filename, "w") do fid
    fid["U_targets"] = U_targets
    fid["exact_x"] = exact_x
    fid["obj_exact_x"] = obj_exact_x
    fid["tssos_glob_obj_min"] = tssos_glob_obj_min
    fid["norm_U_target_minus_obtained"] = norm_U_target_minus_obtained
    fid["f_PSU"] = f_PSU
    fid["convergence_test_exact_x"] = convergence_test_exact_x
    fid["convergence_test_min_x"] = convergence_test_min_x
end;